In [25]:
import torch 
import torchaudio
import librosa
import numpy as np 
from transformers import Wav2Vec2Processor, Wav2Vec2Model 

In [26]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h") 

In [27]:
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h") 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval() 

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Wav2Vec2Encoder(
    (pos_conv_embed): Wav2Vec2PositionalConvEmbedding(
  

In [28]:
audio_path = "medi/WhoLettheDogsOut_Medi.mp3"
waveform, sr = librosa.load(audio_path, sr=16000) 
waveform_tensor = torch.tensor(waveform).unsqueeze(0).to(device)
print(waveform_tensor.shape) 

torch.Size([1, 2989802])


In [ ]:
inputs = processor(waveform, sampling_rate=16000, return_tensors="pt", padding=True)
inputs = {key: val.to(device) for key, val in inputs.items()} 
with torch.no_grad():
    outputs = model(**inputs) 
wav2vec_embedding = outputs.last_hidden_state 
print(wav2vec_embedding.shape) 